## Optimización del Mix de Producción Industrial

#### Descripción del Problema:
Se necesita decidir sobre las cantidades a producir de las distintas calidades de azúcar, con el objetivo de maximizar el *beneficio económico*.

El beneficio económico depende de los costos de producción asociados, así como el precio de venta. Así mismo, las capacidades instaladas de la fábrica de azúcar y el azúcar proyectado disponible para producción. Adicionalmente, el nivel de producción puede verse comprometido por el presupuesto asignado para cubrir los costos de producción.
Esta complejidad es aún mayor cuando se introducen restricciones de almacenamiento (bodegas) y demanda cuando no es posible vender toda el azúcar producida.

Este notebook provee una abstracción matemática del objetivo, así como de las restricciones para solucionarlo por medio de *Optimización con Programación Lineal*.

### Formulación del Problema de Optimización

#### Conjuntos y Notaciones:

- Sea $t$ el número de diferentes tipos o calidades de azúcar que es posible producir en la fábrica.
- Sea $T = \{1,...,t\}$ el conjunto de calidades de azúcar.

#### Parámetros
- Sea $PRICE_i$ el Precio de Venta ($/t) de la calidad de azúcar $i$.
- Sea $COST_i$ el Costo de Producción ($/t) asociado a la calidad de azúcar $i$.
- Sea $CAPACITY_i$ la Capacidad de Producción (t) en la fábrica de la calidad de azúcar $i$.
- Sea $RECOVERY$ el % de Recuperación Global de la fábrica.
- Sea $CORE$ el rendimiento de azúcar esperado (kg Azúcar / t de Caña) de la caña que ingresa a la fábrica.
- Sea $CAÑA$ la cantidad de caña total (t) disponible para molienda.
- Sea $BUDGET$ el presupuesto total disponible.

#### Variables de Decisión
- Sea $x_i \in \mathbb{R}^+$, la cantidad de azúcar a producir de la calidad $i$.

#### Función Objetivo
La función objetivo se define como la suma de los beneficios económicos de las calidades de azúcar seleccionadas para producir. Buscamos maximizar esta cantidad sujeta a la restricción de presupuesto, azúcar disponible y capacidades de producción en la fábrica.

$\begin{align}
\max\limits_{\forall i \in T} \sum_{i=1}^{t} (PRICE_i - COST_i) x_i
\end{align}$

Sujeta a:

$\begin{align}
\sum_{i=1}^{t} COST_i x_i \leq BUDGET, \forall i \in T \\
\sum_{i=1}^{t} x_i \leq \dfrac{CAÑA * CORE * RECOVERY}{1000}, \forall i \in T \\
x_i \leq CAPACITY_i, \forall i \in T \\
x_i \in \mathbb{R}^+, \forall i \in T
\end{align}$


In [1]:
# librerías de optimización
from gurobipy import *
import pandas as pd

### Ejemplo
Suponga que se tienen los siguientes datos de producción en la fábrica de Pantaleón, con sus respectivos precios y costos.

***"Qué calidades y qué cantidades de azúcar debería producir para maximizar el Beneficio Económico total?"***

In [2]:
df = pd.read_csv('datos_azucar.csv')
df['Revenue ($/t)'] = df['Precio ($/t)'] - df['Costo ($/t)']
df

,Tipo Azucar,Precio ($/t),Costo ($/t),Capacidad (t/sem),Revenue ($/t)
0,Blanco,395.69,37.73,9800,357.96
1,Moreno,395.69,38.86,5900,356.83
2,Crudo,343.09,28.94,5600,314.15
3,HTM,415.58,27.40,1800,388.18
4,Refino,420.00,60.61,6400,359.39
5,Miel B,300.00,28.94,300,271.06


#### Parámetros del Problema de Optimización

In [3]:
# products data
tiposAzucar, precio, costo, capacidad, revenue = multidict({
    'Blanco': df.iloc[0,1:].values,
    'Moreno': df.iloc[1,1:].values,
    'Crudo': df.iloc[2,1:].values,
    'HTM': df.iloc[3,1:].values,
    'Refino': df.iloc[4,1:].values,
    'MielB': df.iloc[5,1:].values,
})

# sugar available
core_sampler = 110.00
total_caña = 205000.00
recuperacion = 0.82

# budget available (expressed as a fraction of the cost needed to exploit all the capacity)
budget = 1206998.0*0.75

#### Definición del Modelo de Optimización

In [4]:
# create factory new model
f = Model("FactoryPSA")

# create decision variables for the products to make
cantidad = f.addVars(tiposAzucar, name="cantidad")

Set parameter Username


#### Restricciones al Problema de Optimización


In [5]:
# capacity constraints
cap_res = f.addConstrs(((cantidad[t] <= capacidad[t]) for t in tiposAzucar), name='C')

# total sugar available constraint
tot_res = f.addConstr((sum(cantidad[t] for t in tiposAzucar) <= core_sampler*total_caña*recuperacion/1000), name='T')

# budget constraint
budget_res = f.addConstr((sum(costo[t]*cantidad[t] for t in tiposAzucar) <= budget), name='B')

#### Función Objetivo
Maximizar el Beneficio económico basado en precio de venta y costos de producción.

In [6]:
# the objective is to maximize total profit
f.setObjective(cantidad.prod(revenue),GRB.MAXIMIZE)

#### Optimización

In [7]:
f.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 6 columns and 18 nonzeros
Model fingerprint: 0x55945743
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [3e+02, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 9e+05]
Presolve removed 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.1778364e+06   1.669100e+04   0.000000e+00      0s
       1    6.6820315e+06   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.682031530e+06


In [8]:
# display optimal values of decision variables
for v in f.getVars():
    if(abs(v.x)>1e-6):
        print(v.varName, round(v.x,2))

# display optimal total profit value
print('total revenue ($)', round(f.objVal,2))

cantidad[Blanco] 9800.0
cantidad[Moreno] 491.0
cantidad[HTM] 1800.0
cantidad[Refino] 6400.0
total revenue ($) 6682031.53


Restricciones adicionales pueden incluirse, tales como:
- *Bodegas.* Si alguna bodega no tiene espacio para recibir la producción, esto es una limitante sobre la cantidad a producir de una calidad de azúcar específica.
- *Transporte.* Idem anterior.
- *Demanda.* El equipo comercial puede conocer si las cantidades a producir pueden ser vendidas. Esto es una limitante sobre la cantidad a producir de una calidad de azúcar específica.